In [10]:
# Importações
import pandas as pd
from datetime import datetime
from pathlib import Path
import sqlite3 as sql

# Coleta de Dados do Site do IBGE

In [11]:
# Coletando dados
url = 'https://ibge.gov.br/explica/codigos-dos-municipios.php#RJ'

#pd.read_html(url)[19] #Filtrando por contagem de Tables
DadosIBGE = pd.DataFrame(pd.read_html(url, match="Municípios do Rio de Janeiro")[0])
DadosIBGE

,Municípios do Rio de Janeiro,Códigos
0,Angra dos Reis,3300100
1,Aperibé,3300159
2,Araruama,3300209
3,Areal,3300225
4,Armação dos Búzios,3300233
...,...,...
87,Três Rios,3306008
88,Valença,3306107
89,Varre-Sai,3306156
90,Vassouras,3306206


In [12]:
# Renomarea Colunas
DadosIBGE = DadosIBGE.rename(columns={'Municípios do Rio de Janeiro':'nmMunic', 'Códigos':'codMunic'})
DadosIBGE

,nmMunic,codMunic
0,Angra dos Reis,3300100
1,Aperibé,3300159
2,Araruama,3300209
3,Areal,3300225
4,Armação dos Búzios,3300233
...,...,...
87,Três Rios,3306008
88,Valença,3306107
89,Varre-Sai,3306156
90,Vassouras,3306206


In [13]:
# Alterando nome do index
DadosIBGE.index.name = 'idMunic'
DadosIBGE

,nmMunic,codMunic
idMunic,,
0,Angra dos Reis,3300100
1,Aperibé,3300159
2,Araruama,3300209
3,Areal,3300225
4,Armação dos Búzios,3300233
...,...,...
87,Três Rios,3306008
88,Valença,3306107
89,Varre-Sai,3306156


In [14]:
# Ajustando para começar de 1
DadosIBGE.index = DadosIBGE.index + 1
DadosIBGE

,nmMunic,codMunic
idMunic,,
1,Angra dos Reis,3300100
2,Aperibé,3300159
3,Araruama,3300209
4,Areal,3300225
5,Armação dos Búzios,3300233
...,...,...
88,Três Rios,3306008
89,Valença,3306107
90,Varre-Sai,3306156


In [15]:
# Adicionando coluna de data e hora de carga

data_carga = datetime.today().strftime('%d/%m/%Y %H:%M')
DadosIBGE['dtCarga'] = data_carga
DadosIBGE

,nmMunic,codMunic,dtCarga
idMunic,,,
1,Angra dos Reis,3300100,25/10/2023 13:47
2,Aperibé,3300159,25/10/2023 13:47
3,Araruama,3300209,25/10/2023 13:47
4,Areal,3300225,25/10/2023 13:47
5,Armação dos Búzios,3300233,25/10/2023 13:47
...,...,...,...
88,Três Rios,3306008,25/10/2023 13:47
89,Valença,3306107,25/10/2023 13:47
90,Varre-Sai,3306156,25/10/2023 13:47


# Criando o Banco de Dados ODS

In [16]:
# Manupulando sistema de arquivos
caminho = Path('C:\\Users\\LBorba-GD\\Meu Drive\\Academy\\ClaudioBonel\\Engenharia de Dados\\')

In [17]:
# Criando Banco de Dados OSD
BDODS = caminho / 'pascoaODS.db'
BDDW = caminho / 'pascoaDW.db'


# Criando Validação o caminho existe
if caminho.exists():
    # Validando se o banco de dados existe
    if (BDODS.exists() and BDDW.exists()):
        print('Banco de Dados já existem!')
    else:
        BDODS.touch()
        BDDW.touch()
        print('Banco de Dados criados com sucesso!')
else:
    print('Endereço não existe! Favor Verificar')

Banco de Dados já existem!


# Manipulando os Bancos de Dados Criados

In [18]:
# Conectando no BDODS
conexaoODS = sql.connect(BDODS)

# Criação da tabela tbLogMunic e carregar os dados do DF DadosIBGE
DadosIBGE.to_sql('tbLogMunic',conexaoODS, if_exists="append")

# Confirmando a Transação
conexaoODS.commit()

# Fechar a conexão
conexaoODS.close()

print('Carga do BDODS concluída')

Carga do BDODS concluída


In [21]:
# Conectando no BDDW
conexaoDW = sql.connect(BDDW)

# Filtrando Dados a serem enviados ao DW
DadosIBGE = DadosIBGE[['nmMunic','codMunic']]

# Criação da tabela dMunicipio e carregar os dados do DF DadosIBGE
DadosIBGE.to_sql('dMunicipio',conexaoDW, if_exists="replace")

# Confirmando a Transação
conexaoDW.commit()

# Fechar a conexão
conexaoDW.close()

print('Carga do BDDW concluída')

Carga do BDDW concluída


,nmMunic,codMunic
idMunic,,
1,Angra dos Reis,3300100
2,Aperibé,3300159
3,Araruama,3300209
4,Areal,3300225
5,Armação dos Búzios,3300233
...,...,...
88,Três Rios,3306008
89,Valença,3306107
90,Varre-Sai,3306156
